# ![.right](https://lh6.ggpht.com/wUrTIhpHPVqw_VPCdhbSiF5TXlBlLyRHdH1gsII_y5NkgYKzbbj7cC2l6AGoVq-JN0U=w100 "MSDS 7331 Data Mining - Project 1")


# Project 3: BEER ME! 

### a recommendation system for those who are thirsty but don't know it yet

---



# Team Members

- Chris Woodard
- Claire Chu
- Nathan Mowat
- Bill Kerneckel

---


# Rubic

- [Business Understanding](#dp1) 

- [Data Understanding 1](#dp2)

- [Data Understanding 2](#me1)

- [Modeling and Evaluation 1](#me2)

- [Modeling and Evaluation 2](#me3)

- [Modeling and Evaluation 3](#me4)

- [Modeling and Evaluation 4](#me5)

- [Deployment](#d)

- [Exceptional Work](#ew)

- [Appendix](#a)

<hr>



# OBJECTIVE: 
We found a comprehensive list of beer reviews from beeradvocate and ratebeer. We want to use these reviews to generate a recommendation system that will suggest potential beers that a user might like based on their previous beer ratings and reviews:
- Establish a solid distance metric to determine the similiarity amongst different users
- Utilize this metric to recommend other beers that a user might like

We hope that the collaborative filtering on a custom recommendation system completed in this notebook will help make beer drinkers thirsty for beers they have not tried yet.

source: https://data.world/socialmediadata/beeradvocate

source: http://nbviewer.jupyter.org/gist/glamp/20a18d52c539b87de2af

source: https://snap.stanford.edu/data/web-BeerAdvocate.html

source: https://github.com/yhat/Beer-Rec-Flask/blob/master/README.md

# Business Understanding

---

<span style="color:red">10 Points - Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?</span>

This data was collected from Beeradvocate.com and ratebeer.com from January 1998 - November 2011. Both websites are used by beer drinkers to not only stay informed about the latest in beer news, but to also track and review the beers each user has tried. As an avid beer drinker myself, these websites are useful to me in that they help me avoid re-ordering beers i've previously tried and did not like. This helps me save money and stomach space when i'm ordering drinks at the bar. 

The dataset we are analyzing has a total of 13 columns including: 

-brewery Id
-brewery name
-review time
-overall review rating
-review of aroma
-review of appears
-username of reviewer
-beer style
-review palate
-review taste
-beer name
-alcohol percentage by volume
-beer ID

A good algorithm will be able to suggest a beer that the user will drink and rate at least a 4 or above in their overall rating (we chose 4 because this is the median review rating). If a user reviews the suggested beer at less than 4, our recommender system is not effective. 

Statistically, we will compare amongst the models RMSE. We want to choose the model with the lowest RMSE. We also plan to look at the precision/recall metrics and factor it into our "best model" choice.

# Data Understanding

---

<span style="color:red"> 10 Points - Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file. Verify data quality: Are there missing values? Duplicate data? Outliers? Are those mistakes? How do you deal with these problems?</span>

Insert content here!.......

In [85]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
#load python libaries
import numpy as np
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as pl
import seaborn as sns
import glob
import sys
import os
import warnings
import ggplot
import datetime
import time
from datetime import date

In [2]:
%matplotlib inline

In [3]:
#load data file
beerDF = pd.read_csv('beer_reviews.csv') 

#you'll have to change the file path to your working directory


In [5]:
beerDF.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [6]:
summary = beerDF.describe()
summary = summary.transpose()
summary

,count,mean,std,min,25%,50%,75%,max
brewery_id,1586614.0,3.130099e+03,5.578104e+03,1.000000e+00,1.430000e+02,4.290000e+02,2.372000e+03,2.800300e+04
review_time,1586614.0,1.224089e+09,7.654427e+07,8.406720e+08,1.173224e+09,1.239203e+09,1.288568e+09,1.326285e+09
review_overall,1586614.0,3.815581e+00,7.206219e-01,0.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
review_aroma,1586614.0,3.735636e+00,6.976167e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_appearance,1586614.0,3.841642e+00,6.160928e-01,0.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_palate,1586614.0,3.743701e+00,6.822184e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_taste,1586614.0,3.792860e+00,7.319696e-01,1.000000e+00,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
beer_abv,1518829.0,7.042387e+00,2.322526e+00,1.000000e-02,5.200000e+00,6.500000e+00,8.500000e+00,5.770000e+01
beer_beerid,1586614.0,2.171279e+04,2.181834e+04,3.000000e+00,1.717000e+03,1.390600e+04,3.944100e+04,7.731700e+04


In [7]:
#what is the most popular beer?
beerDF.beer_name.describe()

count           1586614
unique            56857
top       90 Minute IPA
freq               3290
Name: beer_name, dtype: object

In [75]:
#what are the top 10 most popular beers?
from collections import Counter
topbeer = Counter(beerDF.beer_name)
topbeer.most_common(10)

[('90 Minute IPA', 3290),
 ('India Pale Ale', 3130),
 ('Old Rasputin Russian Imperial Stout', 3111),
 ('Sierra Nevada Celebration Ale', 3000),
 ('Two Hearted Ale', 2728),
 ('Stone Ruination IPA', 2704),
 ('Arrogant Bastard Ale', 2704),
 ('Sierra Nevada Pale Ale', 2587),
 ('Stone IPA (India Pale Ale)', 2575),
 ('Pliny The Elder', 2527)]

In [157]:
#which reviewer has the most reviews?

toprevs = Counter(beerDF.review_profilename)
toprevs.most_common(100)

[('northyorksammy', 5817),
 ('BuckeyeNation', 4661),
 ('mikesgroove', 4617),
 ('Thorpe429', 3518),
 ('womencantsail', 3497),
 ('NeroFiddled', 3488),
 ('ChainGangGuy', 3471),
 ('brentk56', 3357),
 ('Phyl21ca', 3179),
 ('WesWes', 3168),
 ('oberon', 3128),
 ('feloniousmonk', 3080),
 ('akorsak', 3010),
 ('BEERchitect', 2946),
 ('Gueuzedude', 2938),
 ('jwc215', 2735),
 ('russpowell', 2696),
 ('TheManiacalOne', 2659),
 ('Gavage', 2630),
 ('zeff80', 2622),
 ('Mora2000', 2594),
 ('tempest', 2559),
 ('Wasatch', 2541),
 ('WVbeergeek', 2524),
 ('drabmuh', 2481),
 ('Halcyondays', 2457),
 ('weeare138', 2452),
 ('glid02', 2450),
 ('Viggo', 2433),
 ('ppoitras', 2426),
 ('jdhilt', 2422),
 ('barleywinefiend', 2396),
 ('Bighuge', 2376),
 ('Jason', 2349),
 ('wl0307', 2315),
 ('Gusler', 2297),
 ('emerge077', 2250),
 ('Billolick', 2238),
 ('smcolw', 2224),
 ('rhoadsrage', 2217),
 ('DrJay', 2202),
 ('Zorro', 2198),
 ('TMoney2591', 2169),
 ('MasterSki', 2156),
 ('Kegatron', 2150),
 ('Knapp85', 2143),
 ('Dogb

In [55]:
#investigate any NA
beerDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_abv              1518829 non-null float64
beer_beerid           1586614 non-null int64
dtypes: float64(6), int64(3), object(4)
memory usage: 157.4+ MB


In [64]:
beerDF.brewery_name.isnull().sum().sum()

15

In [68]:
beerDF.review_profilename.isnull().sum().sum()

348

In [67]:
beerDF.beer_abv.isnull().sum().sum()

67785

Since the majority of our null values are in the "beer_abv" column let's drop this column. If this recommendation system were to go into full scale production, we could replace the null values with the median abv of each style of beer. But, since we are limited in time and computing power, we will delete this column for now.

As our next highest amount of null values comes from the "review_profilename" column, we will need to delete the rows with no profilename. Filling all null with the same entry would result in a faulty statistic result since each null entry was probably entered by different users. Since we want to try collaborative filtering based on user preferences, we will assume that a "null" entry in the profilename space is not useful. It seems like our best bet would be to delete these entries.

The column with the lowest number of null values is the "brewery_name" column. 

In [70]:
beerDF.drop('beer_abv', axis=1, inplace=True)
beerDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 12 columns):
brewery_id            1586614 non-null int64
brewery_name          1586599 non-null object
review_time           1586614 non-null int64
review_overall        1586614 non-null float64
review_aroma          1586614 non-null float64
review_appearance     1586614 non-null float64
review_profilename    1586266 non-null object
beer_style            1586614 non-null object
review_palate         1586614 non-null float64
review_taste          1586614 non-null float64
beer_name             1586614 non-null object
beer_beerid           1586614 non-null int64
dtypes: float64(5), int64(3), object(4)
memory usage: 145.3+ MB


In [71]:
beerDF2 = beerDF[beerDF.review_profilename.notnull()]
beerDF2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586266 entries, 0 to 1586613
Data columns (total 12 columns):
brewery_id            1586266 non-null int64
brewery_name          1586251 non-null object
review_time           1586266 non-null int64
review_overall        1586266 non-null float64
review_aroma          1586266 non-null float64
review_appearance     1586266 non-null float64
review_profilename    1586266 non-null object
beer_style            1586266 non-null object
review_palate         1586266 non-null float64
review_taste          1586266 non-null float64
beer_name             1586266 non-null object
beer_beerid           1586266 non-null int64
dtypes: float64(5), int64(3), object(4)
memory usage: 157.3+ MB


In [72]:
beerDF3 = beerDF2[beerDF2.brewery_name.notnull()]
beerDF3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1586251 entries, 0 to 1586613
Data columns (total 12 columns):
brewery_id            1586251 non-null int64
brewery_name          1586251 non-null object
review_time           1586251 non-null int64
review_overall        1586251 non-null float64
review_aroma          1586251 non-null float64
review_appearance     1586251 non-null float64
review_profilename    1586251 non-null object
beer_style            1586251 non-null object
review_palate         1586251 non-null float64
review_taste          1586251 non-null float64
beer_name             1586251 non-null object
beer_beerid           1586251 non-null int64
dtypes: float64(5), int64(3), object(4)
memory usage: 157.3+ MB


In [73]:
#review our new dataset without any null values
summary3 = beerDF3.describe()
summary3 = summary3.transpose()
summary3

,count,mean,std,min,25%,50%,75%,max
brewery_id,1586251.0,3.130381e+03,5.578478e+03,1.0,1.430000e+02,4.290000e+02,2.372000e+03,2.800300e+04
review_time,1586251.0,1.224100e+09,7.654413e+07,840672001.0,1.173236e+09,1.239222e+09,1.288572e+09,1.326285e+09
review_overall,1586251.0,3.815632e+00,7.205920e-01,0.0,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
review_aroma,1586251.0,3.735688e+00,6.975649e-01,1.0,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_appearance,1586251.0,3.841677e+00,6.160621e-01,0.0,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_palate,1586251.0,3.743757e+00,6.821700e-01,1.0,3.500000e+00,4.000000e+00,4.000000e+00,5.000000e+00
review_taste,1586251.0,3.792925e+00,7.319083e-01,1.0,3.500000e+00,4.000000e+00,4.500000e+00,5.000000e+00
beer_beerid,1586251.0,2.171468e+04,2.181901e+04,3.0,1.717000e+03,1.390600e+04,3.944100e+04,7.731700e+04


## Extracting Reviews

In [76]:
#convert name of dataset from beerdf3 to beerdf
beerDF = beerDF3.copy()

In [77]:
beer_1, beer_2 = "90 Minute IPA", "India Pale Ale"

def get_beer_reviews(beer, common_users):
    mask = (beerDF.review_profilename.isin(common_users)) & (beerDF.beer_name==beer)
    reviews = beerDF[mask].sort('review_profilename')
    reviews = reviews[reviews.review_profilename.duplicated()==False]
    return reviews
beer_1_reviews = get_beer_reviews(beer_1, common_reviewers)
beer_2_reviews = get_beer_reviews(beer_2, common_reviewers)

cols = ['beer_name', 'review_profilename', 'review_overall', 'review_aroma', 'review_palate', 'review_taste']
beer_2_reviews[cols].head()

//anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,beer_name,review_profilename,review_overall,review_aroma,review_palate,review_taste
595646,India Pale Ale,1fastz28,3.5,4.0,3.5,3.0
212302,India Pale Ale,49degrees,3.5,3.0,3.0,3.5
1348197,India Pale Ale,99bottles,4.5,4.0,4.0,4.5
1406683,India Pale Ale,9InchNails,3.5,3.5,4.0,3.5
595971,India Pale Ale,ADR,4.0,3.5,3.5,3.0


After reviewing the dataset, we can see that there are a total of 1586251 complete rows of data comprised of 12 total columns. 

-brewery Id: numeric value representing brewery name, type int64 but nominal in nature
-brewery name: string representing brewery name, type object 
-review time: numeric timestamp repesenting time and date review was completed, type int 64 but with timescale
-overall review rating: categorical from 1-5 representing overall rating, type float64 
-review of aroma: categorical from 1-5 representing aroma rating, type float64 
-review of appearance: categorical from 1-5 representing appearance rating, type float64 
-username of reviewer: string representing user name, type object
-beer style: string representing style of beer, type object
-review palate: categorical from 1-5 representing mouthfeel rating, type float64 
-review taste: categorical from 1-5 representing taste rating, type float64 
-beer name: string representing name of beer given by brewery, type object
-beer ID: numeric value representing beer name, type int 64 but nominal in nature

Data quality was verified with the original dataset having 1586614 values including null. After removing the ABV column and eliminating rows with null values, we have 1586251 total rows and 12 columns.

Looking at the summary of the data, it does not appear that there are any duplicates for the data as the system does not allow users to rate a beer more than once. We can see that the average overall review rating was 3.8. As mentioned earlier, we will round this value up to 4 and use this as our benchmark value to determine if our recommendation system is working. (After completing a review, suggested beers should be rated by the viewer at least 4 or higher)

source: https://www.beeradvocate.com/community/threads/how-to-review-a-beer.241156/

# Data Understanding 2

----


<span style="color:red">10 Points - Visualize the any important attributes appropriately. Important: Provide an interpretation for any charts or graphs.</span>

In [81]:
pd.crosstab(index=beerDF["review_overall"],columns="count") 

col_0,count
review_overall,
0.0,7
1.0,10954
1.5,12972
2.0,38206
2.5,58485
3.0,165582
3.5,301743
4.0,582683
4.5,324321


In [151]:
# percentage of each review rating
from pylab import *
figure(1, figsize=(6,6))
ax = axes([0.2, 0.2, 1.3, 1.3])

# The slices will be ordered and plotted counter-clockwise.
labels = '0.0','1.0', '1.5', '2.0', '2.5', '3.0', '3.5', '4.0', '4.5', '5.0'
fracs = [7, 10954, 12972, 38206, 58485, 165582, 301743, 582683, 324321, 91298]
explode = [0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.1, 0]

pie(fracs, explode=explode, labels=labels,
                autopct='%1.1f%%', shadow=False, startangle=160)

title('Percentage of review ratings', bbox={'facecolor':'0.9', 'pad':5})

#show()

In [84]:
#plot the top ten reviewers by number of reviews
s = pd.Series(np.random.choice(list(string.uppercase), 1000))
counts = beerDF.review_profilename.value_counts()
ax = counts.iloc[:10].plot(kind="barh")
ax.invert_yaxis()

#source: http://stackoverflow.com/questions/28022227/sorted-bar-charts-with-pandas-matplotlib-or-seaborn

NameError: name 'string' is not defined

In [ ]:
#some more graphs and pretty pictures?

# Modeling and Evaluation 1
---

<span style="color:red">10 Points - Train and adjust parameters</span>

We used this section as a proof of concept to test our theory on a subset of data.

## Calculate Euclidean Distances 

In [86]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from scipy.stats.stats import pearsonr


ALL_FEATURES = ['review_overall', 'review_aroma', 'review_palate', 'review_taste']
def calculate_similarity(beer1, beer2):
    # find common reviewers
    beer_1_reviewers = beerDF[beerDF.beer_name==beer1].review_profilename.unique()
    beer_2_reviewers = beerDF[beerDF.beer_name==beer2].review_profilename.unique()
    common_reviewers = set(beer_1_reviewers).intersection(beer_2_reviewers)

    # get reviews
    beer_1_reviews = get_beer_reviews(beer1, common_reviewers)
    beer_2_reviews = get_beer_reviews(beer2, common_reviewers)
    dists = []
    for f in ALL_FEATURES:
        dists.append(euclidean_distances(beer_1_reviews[f], beer_2_reviews[f])[0][0])
    
    return dists

calculate_similarity(beer_1, beer_2)

[27.308423608842748, 27.37243138634199, 26.177280225416848, 29.828677476549309]

## Calculate the Similarity of a Set of Beers

In [150]:
# calculate only a subset for the demo
beers = ["Dale's Pale Ale", 
         "Sierra Nevada Pale Ale", 
         "Michelob Ultra",
         "Natural Light", 
         "Bud Light", 
         "Fat Tire Amber Ale", 
         "Coors Light",
         "Blue Moon Belgian White", 
         "60 Minute IPA", 
         "Guinness Draught"]
# calculate everything for real production
#beers = beerDF.beer_name.unique()

simple_distances = []
for beer1 in beers:
    print ("starting", beer1)
    for beer2 in beers:
        if beer1 != beer2:
            row = [beer1, beer2] + calculate_similarity(beer1, beer2)
            simple_distances.append(row)

starting Dale's Pale Ale
starting Sierra Nevada Pale Ale
starting Michelob Ultra
starting Natural Light
starting Bud Light
starting Fat Tire Amber Ale
starting Coors Light
starting Blue Moon Belgian White
starting 60 Minute IPA
starting Guinness Draught


## Inspect the Results

In [153]:
cols = ["beer1", "beer2", "overall_dist", "aroma_dist", "palate_dist", "taste_dist"]
simple_distances = pd.DataFrame(simple_distances, columns=cols)
simple_distances.tail()

,beer1,beer2,overall_dist,aroma_dist,palate_dist,taste_dist
85,Guinness Draught,Bud Light,43.854874,42.388088,45.694639,44.516851
86,Guinness Draught,Fat Tire Amber Ale,23.611438,21.095023,24.591665,22.632941
87,Guinness Draught,Coors Light,41.249242,38.337319,43.182751,40.416581
88,Guinness Draught,Blue Moon Belgian White,27.285527,22.169799,25.475478,23.780244
89,Guinness Draught,60 Minute IPA,31.200962,32.638168,32.190837,34.452866


## Allow Users to Customize the Weights

In [154]:
def calc_distance(dists, beer1, beer2, weights):
    mask = (dists.beer1==beer1) & (dists.beer2==beer2)
    row = dists[mask]
    row = row[['overall_dist', 'aroma_dist', 'palate_dist', 'taste_dist']]
    dist = weights * row
    return dist.sum(axis=1).tolist()[0]

weights = [2, 1, 1, 1] #Give your ratings here
print (calc_distance(simple_distances, "Fat Tire Amber Ale", "Dale's Pale Ale", weights))
print (calc_distance(simple_distances, "Fat Tire Amber Ale", "Michelob Ultra", weights))

86.40810158004888
152.68128609273793


## Recommend beers similar to my_beer

In [156]:
my_beer = "Dale's Pale Ale"
results = []
for b in beers:
    if my_beer!=b:
        results.append((my_beer, b, calc_distance(simple_distances, my_beer, b, weights)))
sorted(results, key=lambda x: x[2])

[("Dale's Pale Ale", 'Sierra Nevada Pale Ale', 84.76009505678336),
 ("Dale's Pale Ale", '60 Minute IPA', 86.14915895628911),
 ("Dale's Pale Ale", 'Fat Tire Amber Ale', 86.40810158004888),
 ("Dale's Pale Ale", 'Blue Moon Belgian White', 98.32976308091568),
 ("Dale's Pale Ale", 'Guinness Draught', 120.85629336353787),
 ("Dale's Pale Ale", 'Natural Light', 121.55801164144607),
 ("Dale's Pale Ale", 'Michelob Ultra', 149.23623122808397),
 ("Dale's Pale Ale", 'Coors Light', 181.03982876411666),
 ("Dale's Pale Ale", 'Bud Light', 196.80179707789142)]

In [155]:
my_beer = "Bud Light"
results = []
for b in beers:
    if my_beer!=b:
        results.append((my_beer, b, calc_distance(simple_distances, my_beer, b, weights)))
sorted(results, key=lambda x: x[2])

[('Bud Light', 'Natural Light', 64.5003477938315),
 ('Bud Light', 'Michelob Ultra', 74.33645512278882),
 ('Bud Light', 'Coors Light', 100.01552386270659),
 ('Bud Light', 'Blue Moon Belgian White', 184.3851227577757),
 ('Bud Light', "Dale's Pale Ale", 196.80179707789142),
 ('Bud Light', 'Fat Tire Amber Ale', 198.59291387382788),
 ('Bud Light', 'Guinness Draught', 220.30932592966633),
 ('Bud Light', '60 Minute IPA', 262.58747519219816),
 ('Bud Light', 'Sierra Nevada Pale Ale', 281.0170600879314)]

These results make sense since we would assume from the style of beer that Dale's Pale Ale and Sierra Nevada Pale Ale are the most similiar and that Bud Light and Natural Light are the most similar based on the options available. Note that these result are only pulling distances from our subset of 10 that we setup earlier. Notice that we also used "euclidean distance" above. we will work on looking at models that use different distance measures and compare them against each other. 

# Modeling and Evaluation 2 - MUST USE PYTHON 2 KERNAL
---

<span style="color:red">10 Points - Evaluate and Compare</span>

Please note: the use of graphlab here requires an api key. Here we will do a rough cut of the a graphlab 5 line recommender system. 

In [5]:
# This is a well known graphlab example that builds a recommendation system in 5 lines of code
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import graphlab as gl

data = gl.SFrame.read_csv("beer_reviews.csv", 
                          column_type_hints={"review_overall":int})
data = data.dropna()
model = gl.recommender.create(data, user_id="review_profilename", item_id="beer_name", target="review_overall")
results = model.recommend(users=None, k=5)
model.save("my_model")

results.head() # the recommendation output

Finished parsing file C:\Users\natha_000\beer_reviews.csv

Parsing completed. Parsed 100 lines in 0.570646 secs.

Read 453422 lines. Lines per second: 608467

Finished parsing file C:\Users\natha_000\beer_reviews.csv

Parsing completed. Parsed 1586614 lines in 1.6948 secs.

...

recommendations finished on 29000/32909 queries. users per second: 969.916

recommendations finished on 30000/32909 queries. users per second: 969.952

recommendations finished on 31000/32909 queries. users per second: 969.195

recommendations finished on 32000/32909 queries. users per second: 965.937

review_profilename,beer_name,score,rank
stcules,Pioneer Imperial AmericanBrown Ale ...,3.60449393341,1
stcules,Little Magic Pale Ale,3.60449392814,2
stcules,Hackney Gold,3.60449392069,3
stcules,Olympia Beer,3.60449391938,4
stcules,Kryptonite,3.60449391306,5
johnmichaelsen,Pioneer Imperial AmericanBrown Ale ...,3.60410687889,1
johnmichaelsen,Little Magic Pale Ale,3.60410687367,2
johnmichaelsen,Hackney Gold,3.60410686621,3
johnmichaelsen,Olympia Beer,3.60410686487,4
johnmichaelsen,Kryptonite,3.60410685926,5


In the above model creation, we have found the top five highest ranking items for each user. Two users are shown with their corresponding highest ranking items (that they have not rated). It is interesting to note that both users were suggested the same beers. We must assume that both of these users have the same beer tastes and preferences.

In [187]:
####trying to look up a common review to confirm they have similar tastes ....

#find a beer they both reviewed
stcules_revs = beerDF[beerDF.review_profilename =='stcules'].beer_name.unique()
johnmic_revs = beerDF[beerDF.review_profilename =='johnmichaelsen'].beer_name.unique()
common_revs = set(stcules_revs).intersection(johnmic_revs)
print(common_revs)

###getting an error for this section

#find both reviews for 'India Pale Ale'
#IPA_stc = beerDF[beerDF.beer_name == 'India Pale Ale'] & beerDF[beerDF.review_profilename == 'India Pale Ale']
#IPA_joh = johnmic_revs[johnmic_revs.beer_name == 'India Pale Ale']

#print (IPA_stc)
#johnmic_revs.reset_index()


{'India Pale Ale', 'Krušovice Imperial', 'Malheur 10°', 'Prima Pils', 'Weihenstephaner Hefeweissbier', 'Saxo', "Saison D'Erpe-Mere", 'Fantôme De Noel', "Young's Double Chocolate Stout", 'Jubiläums German Pils', 'Bieken', 'Double Dragon', 'Weihenstephaner Korbinian', 'Tripel Karmeliet', 'Porter', 'Fat Tire Amber Ale', 'Shakespeare Oatmeal Stout', 'Piraat Ale', 'Black Mamba', 'Torrey Pines IPA', 'Zinnebir', 'Moinette Blond', 'St. Feuillien Triple', 'Spaten Maibock / Premium Bock', 'Leipziger Gose', 'ViaEmilia', 'K-9 Cruiser Winter Ale', 'Urthel Samaranth', 'Saison', 'Jan De Lichte', 'Noir De Dottignies', 'Gonzo Imperial Porter', 'A Over T (Aromas Over Tongham)', 'Noël Des Géants', 'BrewDog Paradox Smokehead (Batch 015) - Smokehead', 'Trappistes Rochefort 10', 'Cantillon Lou Pepe - Kriek', 'Pilsner', 'Ebulum Elderberry Black Ale', '90 Minute IPA', 'Odell IPA', 'Malheur Brut Noir (Black Chocolate)', 'Poperings Hommel Bier', 'La Rullés Cuvée Meilleurs Voeux (Bière De Gaume)', 'Schneider Ave

In [7]:
data.head()

brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance
10325,Vecchio Birraio,1234817823,1,2.0,2.5
10325,Vecchio Birraio,1235915097,3,2.5,3.0
10325,Vecchio Birraio,1235916604,3,2.5,3.0
10325,Vecchio Birraio,1234725145,3,3.0,3.5
1075,Caldera Brewing Company,1293735206,4,4.5,4.0
1075,Caldera Brewing Company,1325524659,3,3.5,3.5
1075,Caldera Brewing Company,1318991115,3,3.5,3.5
1075,Caldera Brewing Company,1306276018,3,2.5,3.5
1075,Caldera Brewing Company,1290454503,4,3.0,3.5
1075,Caldera Brewing Company,1285632924,4,3.5,5.0


# NEW MODEL STARTS HERE|

In [9]:
from graphlab.toolkits.recommender import item_similarity_recommender

item_item = gl.recommender.item_similarity_recommender.create(data, 
                                  user_id="review_profilename", 
                                  item_id="beer_name", 
                                  target="review_overall",
                                  only_top_k=3,
                                  similarity_type="cosine")

results = item_item.get_similar_items(k=3)
results.head()

Recsys training: model = item_similarity

Warning: Ignoring columns brewery_id, brewery_name, review_time, review_aroma, review_appearance, beer_style, review_palate, review_taste, beer_abv, beer_beerid;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1518829 observations with 32909 users and 44085 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 24.2281s

beer_name,similar,score,rank
Sausa Weizen,Red Moon,1.0,1
Sausa Weizen,Black Horse Black Beer,1.0,2
Sausa Weizen,Sausa Pils,1.0,3
Red Moon,Sausa Weizen,1.0,1
Red Moon,Black Horse Black Beer,1.0,2
Red Moon,Sausa Pils,1.0,3
Black Horse Black Beer,Sausa Weizen,1.0,1
Black Horse Black Beer,Red Moon,1.0,2
Black Horse Black Beer,Sausa Pils,1.0,3
Sausa Pils,Sausa Weizen,1.0,1


In [10]:
train, test = gl.recommender.util.random_split_by_user(data,
                                                    user_id="review_profilename", item_id="beer_name",
                                                    max_num_users=100, item_test_proportion=0.2)

In [12]:
from IPython.display import display
from IPython.display import Image

gl.canvas.set_target('ipynb')


item_item = gl.recommender.item_similarity_recommender.create(train, 
                                  user_id="review_profilename", 
                                  item_id="beer_name", 
                                  target="review_overall",
                                  only_top_k=5,
                                  similarity_type="cosine")

rmse_results = item_item.evaluate(test)

Recsys training: model = item_similarity

Warning: Ignoring columns brewery_id, brewery_name, review_time, review_aroma, review_appearance, beer_style, review_palate, review_taste, beer_abv, beer_beerid;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1518062 observations with 32901 users and 44079 items.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 25.667s


Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.120689655172 | 0.0491763156412 |
|   2    | 0.0862068965517 | 0.0528058291245 |
|   3    | 0.0862068965517 | 0.0632469936005 |
|   4    | 0.0905172413793 | 0.0665055548236 |
|   5    | 0.0896551724138 | 0.0715085379534 |
|   6    | 0.0862068965517 | 0.0727489086112 |
|   7    | 0.0812807881773 | 0.0750070402084 |
|   8    | 0.0797413793103 | 0.0776523414715 |
|   9    | 0.0785440613027 | 0.0818270622619 |
|   10   | 0.0775862068966 | 0.0910161508684 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 3.710958263148474)

Per User RMSE (best)
+--------------------+-------+----------------+
| review_profilename | count |      rmse      |
+--------------------+-------+----------------+
|     AxelTheRed     |   1   | 0.142133593559 |
+-------------

In [13]:
print rmse_results.viewkeys()
print rmse_results['rmse_by_item']

dict_keys(['rmse_by_user', 'precision_recall_overall', 'rmse_by_item', 'precision_recall_by_user', 'rmse_overall'])
+--------------------------------+-------+---------------+
|           beer_name            | count |      rmse     |
+--------------------------------+-------+---------------+
|   XS Imperial India Pale Ale   |   1   | 3.97283652094 |
| Summit 90/- Scottish Style Ale |   1   | 2.97557458684 |
| Stoudt's Blonde Double MaiBock |   1   | 3.99840310128 |
|  White Birch Oak Aged Tripel   |   1   | 3.99163908958 |
|         Amarillo Amber         |   1   |      5.0      |
| Duck-Rabbit Wee Heavy Scot...  |   1   | 2.94532305002 |
|   Dark Horse Boffo Brown Ale   |   1   |      4.0      |
|  El Camino (Un)Real Black Ale  |   1   |      3.0      |
|         Smoked & Oaked         |   1   | 3.92770607557 |
|           Makeweight           |   1   |      4.0      |
+--------------------------------+-------+---------------+
[692 rows x 3 columns]
Note: Only the head of the SFrame i

In [14]:
rmse_results['rmse_by_user']

review_profilename,count,rmse
SchmichaelJ,16,3.69939415274
iddqd,1,3.0
DonaCake,1,5.0
mnmorency,1,4.0
VirtualAsylum,1,1.0
davidvaz,2,1.0
psycle,2,3.16227766017
JEngler,1,3.0
jaymp27,2,3.16227766017
GeorgeAldi,4,3.97675551527


In [15]:
rmse_results['precision_recall_by_user']

review_profilename,cutoff,precision,recall,count
2oose,1,0.0,0.0,3
2oose,2,0.0,0.0,3
2oose,3,0.0,0.0,3
2oose,4,0.0,0.0,3
2oose,5,0.0,0.0,3
2oose,6,0.0,0.0,3
2oose,7,0.0,0.0,3
2oose,8,0.0,0.0,3
2oose,9,0.0,0.0,3
2oose,10,0.0,0.0,3


# Modeling and Evaluation 3
---

In [16]:
import graphlab.aggregate as agg

# we will be using these aggregations
agg_list = [agg.AVG('precision'),agg.STD('precision'),agg.AVG('recall'),agg.STD('recall')]

# apply these functions to each group (we will group the results by 'k' which is the cutoff)
# the cutoff is the number of top items to look for see the following URL for the actual equation
# https://dato.com/products/create/docs/generated/graphlab.recommender.util.precision_recall_by_user.html#graphlab.recommender.util.precision_recall_by_user
rmse_results['precision_recall_by_user'].groupby('cutoff',agg_list)

# the groups are not sorted

cutoff,Avg of precision,Stdv of precision,Avg of recall,Stdv of recall
16,0.0743534482759,0.149737961021,0.110124263367,0.209867676922
10,0.0775862068966,0.146267472812,0.0910161508684,0.202673162242
36,0.0493295019157,0.0996272191046,0.144604925792,0.240462294898
26,0.0583554376658,0.119562168711,0.129850292884,0.236375848164
41,0.0445752733389,0.0898027462966,0.146878904245,0.240558957227
3,0.0862068965517,0.170196470057,0.0632469936005,0.195306723899
1,0.120689655172,0.325766269443,0.0491763156412,0.190190932573
6,0.0862068965517,0.155491370787,0.0727489086112,0.195245798534
11,0.0783699059561,0.150470219436,0.0935816762565,0.202632801668
2,0.0862068965517,0.188869847416,0.0528058291245,0.19032109025


## Cross Validated Collaborative Filtering

In [17]:

rec1 = gl.recommender.ranking_factorization_recommender.create(train, 
                                  user_id="review_profilename", 
                                  item_id="beer_name", 
                                  target="review_overall")

rmse_results = rec1.evaluate(test)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 1518062 observations with 32901 users and 44079 items.

Data prepared in: 3.06407s

Training ranking_factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached (hard limit).

Computing final objective value and training RMSE.

Final objective value: 1.12046

Final training RMSE: 0.742239


Precision and recall summary statistics by cutoff
+--------+-----------------+-------------------+
| cutoff |  mean_precision |    mean_recall    |
+--------+-----------------+-------------------+
|   1    |  0.051724137931 | 0.000492967916815 |
|   2    | 0.0431034482759 | 0.000835390701784 |
|   3    | 0.0402298850575 |  0.00117781348675 |
|   4    | 0.0301724137931 |  0.00117781348675 |
|   5    | 0.0310344827586 |  0.00250894938939 |
|   6    | 0.0258620689655 |  0.00250894938939 |
|   7    | 0.0295566502463 |  0.00443461714946 |
|   8    | 0.0258620689655 |  0.00443461714946 |
|   9    | 0.0268199233716 |  0.00479288679042 |
|   10   | 0.0258620689655 |  0.00492758506628 |
+--------+-----------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.6801645740116268)

Per User RMSE (best)
+--------------------+-------+----------------+
| review_profilename | count |      rmse      |
+--------------------+-------+----------------+
|       bdawe        |   1   | 0.

In [18]:
rmse_results['precision_recall_by_user'].groupby('cutoff',[agg.AVG('precision'),agg.STD('precision'),agg.AVG('recall'),agg.STD('recall')])

cutoff,Avg of precision,Stdv of precision,Avg of recall,Stdv of recall
16,0.020474137931,0.0719081923306,0.00643646714737,0.0222409188275
10,0.0258620689655,0.0882010728443,0.00492758506628,0.0165125030902
36,0.0210727969349,0.066720592978,0.0253576581332,0.0774131758715
26,0.0212201591512,0.0707656316815,0.0107368986666,0.0321769507631
41,0.0193439865433,0.0605550883095,0.0258806043807,0.0778988931479
3,0.0402298850575,0.164672974505,0.00117781348675,0.00478665573676
1,0.051724137931,0.221469527218,0.000492967916815,0.00234024416274
6,0.0258620689655,0.0968949155177,0.00250894938939,0.0101978913813
11,0.025078369906,0.0857353876563,0.00518113476202,0.0172104735391
2,0.0431034482759,0.168269149103,0.000835390701784,0.00318196175368


In [19]:

rec1 = gl.recommender.ranking_factorization_recommender.create(train, 
                                  user_id="review_profilename", 
                                  item_id="beer_name", 
                                  target="review_overall",
                                  num_factors=16,                 # override the default value
                                  regularization=1e-02,           # override the default value
                                  linear_regularization = 1e-3)   # override the default value

rmse_results = rec1.evaluate(test)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 1518062 observations with 32901 users and 44079 items.

Data prepared in: 3.01997s

Training ranking_factorization_recommender for recommendations.

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.718394

Final training RMSE: 0.541016


Precision and recall summary statistics by cutoff
+--------+-----------------+-------------------+
| cutoff |  mean_precision |    mean_recall    |
+--------+-----------------+-------------------+
|   1    | 0.0344827586207 | 0.000342422784969 |
|   2    | 0.0258620689655 | 0.000595972480709 |
|   3    | 0.0344827586207 |  0.00107309354154 |
|   4    | 0.0344827586207 |  0.00143136318249 |
|   5    | 0.0413793103448 |   0.002483196887  |
|   6    | 0.0402298850575 |  0.00413946809535 |
|   7    | 0.0394088669951 |  0.00449773773631 |
|   8    | 0.0366379310345 |  0.00458661082553 |
|   9    | 0.0344827586207 |  0.00467548391476 |
|   10   | 0.0344827586207 |  0.00489905527985 |
+--------+-----------------+-------------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 0.504521213322226)

Per User RMSE (best)
+--------------------+-------+-----------------+
| review_profilename | count |       rmse      |
+--------------------+-------+-----------------+
|      columbo       |   1   | 

<span style="color:red">10 Points - Summarize the Ramifications</span>

Insert Content Here!

# Modeling and Evaluation 4
---

<span style="color:red">20 Points - Summarize the Ramifications. </span>

Insert Content Here!

# COMPARISON - Is this better than the item item matrix?

In [20]:
comparison = gl.recommender.util.compare_models(test, [item_item, rec1])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.120689655172 | 0.0491763156412 |
|   2    | 0.0862068965517 | 0.0528058291245 |
|   3    | 0.0862068965517 | 0.0632469936005 |
|   4    | 0.0905172413793 | 0.0665055548236 |
|   5    | 0.0896551724138 | 0.0715085379534 |
|   6    | 0.0862068965517 | 0.0727489086112 |
|   7    | 0.0812807881773 | 0.0750070402084 |
|   8    | 0.0797413793103 | 0.0776523414715 |
|   9    | 0.0785440613027 | 0.0818270622619 |
|   10   | 0.0775862068966 | 0.0910161508684 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

('\nOverall RMSE: ', 3.710958263148474)

Per User RMSE (best)
+--------------------+-------+----------------+
| review_profilename | count |      rmse      |
+--------------------+-------+----------------+
|     AxelTheRed     |   1   | 0.1

In [21]:
comparisonstruct = gl.compare(test,[item_item, rec1])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-----------------+-----------------+
| cutoff |  mean_precision |   mean_recall   |
+--------+-----------------+-----------------+
|   1    |  0.120689655172 | 0.0491763156412 |
|   2    | 0.0862068965517 | 0.0528058291245 |
|   3    | 0.0862068965517 | 0.0632469936005 |
|   4    | 0.0905172413793 | 0.0665055548236 |
|   5    | 0.0896551724138 | 0.0715085379534 |
|   6    | 0.0862068965517 | 0.0727489086112 |
|   7    | 0.0812807881773 | 0.0750070402084 |
|   8    | 0.0797413793103 | 0.0776523414715 |
|   9    | 0.0785440613027 | 0.0818270622619 |
|   10   | 0.0775862068966 | 0.0910161508684 |
+--------+-----------------+-----------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+-----------------+-------------------+
| cutoff |  mean_precision |    mean_recall    |
+--------+-----------------+-------------------+
|   1    | 0.0

In [22]:
gl.show_comparison(comparisonstruct,[item_item, rec1])

In [ ]:
params = {'user_id': 'user', 
          'item_id': 'movie', 
          'target': 'rating',
          'num_factors': [8, 12, 16, 24, 32], 
          'regularization':[0.001] ,
          'linear_regularization': [0.001]}

job = gl.model_parameter_search.create( (train,test),
        gl.recommender.ranking_factorization_recommender.create,
        params,
        max_models=5,
        environment=None)

# also note thatthis evaluator also supports sklearn
# https://dato.com/products/create/docs/generated/graphlab.toolkits.model_parameter_search.create.html?highlight=model_parameter_search

In [ ]:

job.get_status()

In [ ]:
job_result = job.get_results()

job_result.head()

In [ ]:

bst_prms = job.get_best_params()
bst_prms

In [ ]:
models = job.get_models()
models

In [ ]:
comparisonstruct = gl.compare(test,models)
gl.show_comparison(comparisonstruct,models)

In [ ]:
models[1]

# Deployment
---


<span style="color:red">10 Points - Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling? How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.? </span>

LEFT OVER FROM LAST ASSIGNMENT - All Federal Agencies are required to use and store the data elements defined by Office of Personnel Management (OPM). The data elements are defined in the following OPM website: https://www.opm.gov/policy-data-oversight/pay-leave/salaries-wages/salary-tables/datadictionary.aspx

We strongly recommend all Human Resources and Workforce Planning offices of the United States Government, especially the Office of Personnel Management (OPM) will benefit from the use of the models and analysis contained in the document. 

# Exceptional Work
---

<span style="color:red">10 Points - You have free reign to provide additional analyses or combine analyses.</span>

# Appendix